Every feature depends on the team:
- goals scores (at home and away) DONE
- goals conceded (at home and away) DONE
- goal difference DONE
- current ranking
- average ranking in the last 5 seasons for example
- current streak of games (average number of points gained in the last X games)
- head2head (goals scored, conceded, wins, ties...) DONE
- general win/tie/loses rate of each team
- Number of promotions and relegations ?
- biggest win ?
- longest undefeated streak ?
- day and hour ?

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings

conn = sqlite3.connect('../laliga.sqlite')
df = pd.read_sql_query("SELECT * FROM Matches", conn)
df = df.dropna(axis = 'index' , how = 'all')
display(df.head())

# Close the connection when done
conn.close()

,season,division,matchday,date,time,home_team,away_team,score
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2


In [2]:
df = df.loc[(df['season'] < '2021-2022') & (~df['score'].isnull())]
pd.set_option("display.max_rows", 50)

In [3]:
print(df['home_team'].unique())
print(len(df['home_team'].unique()))

#MÁLAGA
#BURGOS
#Orense
#Sestao

df['home_team'] = df['home_team'].apply(lambda x: 'Atlético Madrid' if x == 'Athletic Madrid'
                                         else 'Real Sociedad' if x == 'Donostia'
                                         else 'CE Europa' if x == 'Catalunya'
                                         else 'Atlético Madrid' if x == 'Ath. Aviación'
                                         else 'Atlético Madrid' if x == 'Atl. Aviación'
                                         else 'UE Lleida' if x == 'Lérida'
                                         else 'UD Almería' if x == 'AD Almería'
                                         else 'Sporting Gijón' if x == 'Real Gijón'
                                         else 'Gimnàstic' if x == 'Gimnástica'
                                         else 'Ciudad Murcia' if x == 'Granada 74 CF'
                                         else 'UD Las Palmas' if x == 'Uni. Las Palmas'
                                         else 'Racing Ferrol' if x == 'Ferrol'
                                         else 'Terrassa FC' if x == 'Tarrasa'
                                         else 'Barcelona B' if x == 'Barcelona Atl.'
                                         else 'RM Castilla' if x == 'Castilla CF'
                                         else x)

df['away_team'] = df['away_team'].apply(lambda x: 'Atlético Madrid' if x == 'Athletic Madrid'
                                         else 'Real Sociedad' if x == 'Donostia'
                                         else 'CE Europa' if x == 'Catalunya'
                                         else 'Atlético Madrid' if x == 'Ath. Aviación'
                                         else 'Atlético Madrid' if x == 'Atl. Aviación'
                                         else 'UE Lleida' if x == 'Lérida'
                                         else 'UD Almería' if x == 'AD Almería'
                                         else 'Sporting Gijón' if x == 'Real Gijón'
                                         else 'Gimnàstic' if x == 'Gimnástica'
                                         else 'Ciudad Murcia' if x == 'Granada 74 CF'
                                         else 'UD Las Palmas' if x == 'Uni. Las Palmas'
                                         else 'Racing Ferrol' if x == 'Ferrol'
                                         else 'Terrassa FC' if x == 'Tarrasa'
                                         else 'Barcelona B' if x == 'Barcelona Atl.'
                                         else 'RM Castilla' if x == 'Castilla CF'
                                        else x)

print(len(df['home_team'].unique()))

df.loc[df['home_team'] == 'Lorca FC']

['Arenas Club' 'Espanyol' 'Real Madrid' 'Donostia' 'Racing' 'Barcelona'
 'Athletic' 'Athletic Madrid' 'Real Unión' 'Catalunya' 'Alavés' 'Valencia'
 'Real Betis' 'Real Oviedo' 'Sevilla FC' 'Hércules CF' 'CA Osasuna'
 'Real Zaragoza' 'Celta de Vigo' 'Ath. Aviación' 'Real Murcia'
 'Atl. Aviación' 'Granada CF' 'Dep. La Coruña' 'Real Sociedad'
 'CD Castellón' 'CE Sabadell ' 'Real Gijón' 'CD Alcoyano' 'Gimnàstic'
 'Atlético Madrid' 'Real Valladolid' 'CD Málaga' 'Lérida'
 'Atlético Tetuán' 'UD Las Palmas' 'Real Jaén CF' 'CyD Leonesa'
 'CD Condal' 'Elche CF' 'RCD Mallorca' 'CD Tenerife' 'Córdoba CF'
 'Pontevedra' 'Levante' 'Sporting Gijón' 'Burgos CF' 'UD Salamanca'
 'Cádiz CF' 'Rayo Vallecano' 'Recr. Huelva' 'AD Almería' 'CD Logroñés'
 'Albacete' 'UE Lleida' 'SD Compostela' 'CP Mérida' 'CF Extremadura'
 'Villarreal' 'Málaga CF' 'CD Numancia' 'Getafe' 'UD Almería' 'Xerez CD'
 'SD Eibar' 'CD Leganés' 'Girona' 'SD Huesca' 'Baracaldo' 'Ferrol'
 'Avilés Ind.' 'SD Ceuta' 'CE Constància' 'Córdoba' '

,season,division,matchday,date,time,home_team,away_team,score
46090,2017-2018,2,1,8/18/17,8:00 PM,Lorca FC,CyD Leonesa,2:0
46117,2017-2018,2,3,9/2/17,6:00 PM,Lorca FC,Rayo Vallecano,0:0
46136,2017-2018,2,5,9/16/17,6:00 PM,Lorca FC,Reus Deportiu,1:1
46159,2017-2018,2,7,9/30/17,8:00 PM,Lorca FC,Cádiz CF,3:0
46184,2017-2018,2,9,10/11/17,10:00 PM,Lorca FC,Real Zaragoza,0:2
46203,2017-2018,2,11,10/21/17,8:00 PM,Lorca FC,CD Tenerife,2:2
46225,2017-2018,2,13,11/4/17,8:00 PM,Lorca FC,Córdoba CF,1:0
46246,2017-2018,2,15,11/18/17,6:00 PM,Lorca FC,CD Lugo,1:2
46269,2017-2018,2,17,12/2/17,8:00 PM,Lorca FC,Real Oviedo,0:2
46301,2017-2018,2,20,12/20/17,8:00 PM,Lorca FC,CA Osasuna,0:1


In [4]:
for team1 in df['away_team'].unique():
    for team2 in df['home_team'].unique():
        print(team1, " : ", team2)

print(df['away_team'].unique())

Atlético Madrid  :  Arenas Club
Atlético Madrid  :  Espanyol
Atlético Madrid  :  Real Madrid
Atlético Madrid  :  Real Sociedad
Atlético Madrid  :  Racing
Atlético Madrid  :  Barcelona
Atlético Madrid  :  Athletic
Atlético Madrid  :  Atlético Madrid
Atlético Madrid  :  Real Unión
Atlético Madrid  :  CE Europa
Atlético Madrid  :  Alavés
Atlético Madrid  :  Valencia
Atlético Madrid  :  Real Betis
Atlético Madrid  :  Real Oviedo
Atlético Madrid  :  Sevilla FC
Atlético Madrid  :  Hércules CF
Atlético Madrid  :  CA Osasuna
Atlético Madrid  :  Real Zaragoza
Atlético Madrid  :  Celta de Vigo
Atlético Madrid  :  Real Murcia
Atlético Madrid  :  Granada CF
Atlético Madrid  :  Dep. La Coruña
Atlético Madrid  :  CD Castellón
Atlético Madrid  :  CE Sabadell 
Atlético Madrid  :  Sporting Gijón
Atlético Madrid  :  CD Alcoyano
Atlético Madrid  :  Gimnàstic
Atlético Madrid  :  Real Valladolid
Atlético Madrid  :  CD Málaga
Atlético Madrid  :  UE Lleida
Atlético Madrid  :  Atlético Tetuán
Atlético Madrid 

In [5]:
# Separate the home score and the away score.
df[['home_score', 'away_score']] = df['score'].str.split(':', expand=True)
df['home_score'] = pd.to_numeric(df['home_score'])
df['away_score'] = pd.to_numeric(df['away_score'])

# Define whether it is a home or away win.
df['result'] = df.apply(lambda row: 'Home Win' if row['home_score'] > row['away_score']
                                        else 'Away Win' if row['home_score'] < row['away_score']
                                        else 'Draw', axis=1)

# Calculate wins, losses, and ties for home and away teams
df['home_win'] = df['result'].apply(lambda x: 1 if x == 'Home Win' else 0)
df['away_win'] = df['result'].apply(lambda x: 1 if x == 'Away Win' else 0)
df['tie'] = df['result'].apply(lambda x: 1 if x == 'Draw' else 0)

df

,season,division,matchday,date,time,home_team,away_team,score,home_score,away_score,result,home_win,away_win,tie
0,1928-1929,1,1,2/10/29,None,Arenas Club,Atlético Madrid,2:3,2,3,Away Win,0,1,0
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2,3,2,Home Win,1,0,0
2,1928-1929,1,1,2/10/29,None,Real Madrid,CE Europa,5:0,5,0,Home Win,1,0,0
3,1928-1929,1,1,2/10/29,None,Real Sociedad,Athletic,1:1,1,1,Draw,0,0,1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2,0,2,Away Win,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47933,2020-2021,2,42,5/30/21,9:00 PM,CD Mirandés,CE Sabadell,0:2,0,2,Away Win,0,1,0
47934,2020-2021,2,42,5/30/21,9:00 PM,Ponferradina,RCD Mallorca,2:2,2,2,Draw,0,0,1
47935,2020-2021,2,42,5/30/21,9:00 PM,Rayo Vallecano,CD Lugo,0:1,0,1,Away Win,0,1,0
47936,2020-2021,2,42,5/30/21,9:00 PM,Real Zaragoza,CD Leganés,0:5,0,5,Away Win,0,1,0


In [6]:
# First, define a function to calculate cumulative stats for each group (i.e., each unique pair of teams).
def calculate_H2H_stats(matches):
    matches = matches.sort_values('season')

     # Calculate rolling cumulative stats for the previous 20 matches
    matches['H2H_wins_home_last_20'] = (matches['home_team'] == matches['home_team'].iloc[0]) * matches['home_win'].rolling(window=20, min_periods=1).sum()
    matches['H2H_wins_away_last_20'] = (matches['away_team'] == matches['away_team'].iloc[0]) * matches['away_win'].rolling(window=20, min_periods=1).sum()
    matches['H2H_ties_last_20'] = matches['tie'].rolling(window=20, min_periods=1).sum()
    matches['H2H_goals_home_last_20'] = (matches['home_team'] == matches['home_team'].iloc[0]) * matches['home_score'].rolling(window=20, min_periods=1).sum()
    matches['H2H_goals_away_last_20'] = (matches['away_team'] == matches['away_team'].iloc[0]) * matches['away_score'].rolling(window=20, min_periods=1).sum()

    return matches

# Apply the function to each group of home and away team pairs.
df = df.groupby(['home_team', 'away_team'], group_keys=False).apply(calculate_H2H_stats)

display(df.loc[(df['home_team'] == "Barcelona") & (df['away_team'] == "Real Madrid")].tail(50))
display(df.loc[(df['away_team'] == "Barcelona") & (df['home_team'] == "Real Madrid")].head(50))
# It works xddd

/tmp/ipykernel_7556/2503260112.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['home_team', 'away_team'], group_keys=False).apply(calculate_H2H_stats)


,season,division,matchday,date,time,home_team,away_team,score,home_score,away_score,result,home_win,away_win,tie,H2H_wins_home_last_20,H2H_wins_away_last_20,H2H_ties_last_20,H2H_goals_home_last_20,H2H_goals_away_last_20
8093,1972-1973,1,5,10/1/72,None,Barcelona,Real Madrid,1:0,1,0,Home Win,1,0,0,11.0,6.0,3.0,35.0,26.0
8410,1973-1974,1,6,10/7/73,None,Barcelona,Real Madrid,0:0,0,0,Draw,0,0,1,10.0,6.0,4.0,30.0,25.0
8951,1974-1975,1,32,5/11/75,None,Barcelona,Real Madrid,0:0,0,0,Draw,0,0,1,10.0,6.0,4.0,28.0,23.0
9102,1975-1976,1,15,12/28/75,None,Barcelona,Real Madrid,2:1,2,1,Home Win,1,0,0,10.0,6.0,4.0,28.0,24.0
9299,1976-1977,1,3,9/19/76,None,Barcelona,Real Madrid,3:1,3,1,Home Win,1,0,0,10.0,6.0,4.0,30.0,25.0
9688,1977-1978,1,12,12/4/77,None,Barcelona,Real Madrid,2:3,2,3,Away Win,0,1,0,10.0,6.0,4.0,32.0,26.0
10069,1978-1979,1,21,2/17/79,None,Barcelona,Real Madrid,2:0,2,0,Home Win,1,0,0,10.0,6.0,4.0,30.0,26.0
10370,1979-1980,1,20,2/10/80,None,Barcelona,Real Madrid,0:2,0,2,Away Win,0,1,0,9.0,7.0,4.0,27.0,27.0
10613,1980-1981,1,13,11/30/80,None,Barcelona,Real Madrid,2:1,2,1,Home Win,1,0,0,10.0,6.0,4.0,26.0,23.0
10948,1981-1982,1,16,12/20/81,None,Barcelona,Real Madrid,3:1,3,1,Home Win,1,0,0,10.0,6.0,4.0,26.0,23.0


,season,division,matchday,date,time,home_team,away_team,score,home_score,away_score,result,home_win,away_win,tie,H2H_wins_home_last_20,H2H_wins_away_last_20,H2H_ties_last_20,H2H_goals_home_last_20,H2H_goals_away_last_20
53,1928-1929,1,11,5/9/29,None,Real Madrid,Barcelona,0:1,0,1,Away Win,0,1,0,0.0,1.0,0.0,0.0,1.0
176,1929-1930,1,18,3/30/30,None,Real Madrid,Barcelona,5:1,5,1,Home Win,1,0,0,1.0,1.0,0.0,5.0,2.0
221,1930-1931,1,9,2/1/31,None,Real Madrid,Barcelona,0:0,0,0,Draw,0,0,1,1.0,1.0,1.0,5.0,2.0
311,1931-1932,1,9,1/31/32,None,Real Madrid,Barcelona,2:0,2,0,Home Win,1,0,0,2.0,1.0,1.0,7.0,2.0
430,1932-1933,1,15,3/5/33,None,Real Madrid,Barcelona,2:1,2,1,Home Win,1,0,0,3.0,1.0,1.0,9.0,3.0
510,1933-1934,1,13,1/28/34,None,Real Madrid,Barcelona,4:0,4,0,Home Win,1,0,0,4.0,1.0,1.0,13.0,3.0
594,1934-1935,1,10,2/3/35,None,Real Madrid,Barcelona,8:2,8,2,Home Win,1,0,0,5.0,1.0,1.0,21.0,5.0
775,1935-1936,1,18,3/22/36,None,Real Madrid,Barcelona,3:0,3,0,Home Win,1,0,0,6.0,1.0,1.0,24.0,5.0
852,1939-1940,1,9,1/28/40,None,Real Madrid,Barcelona,2:1,2,1,Home Win,1,0,0,7.0,1.0,1.0,26.0,6.0
1056,1940-1941,1,21,2/23/41,None,Real Madrid,Barcelona,1:2,1,2,Away Win,0,1,0,7.0,2.0,1.0,27.0,8.0


In [7]:
"""
for team1 in df['away_team'].unique():
    for team2 in df['home_team'].unique():        
        # Create a subset of matches between the two teams
        matches = df[((df['home_team'] == team1) & (df['away_team'] == team2)) |
                            ((df['home_team'] == team2) & (df['away_team'] == team1))]        

        # Calculate wins and ties using home_win and away_win columns
        wins_team1 = matches.apply(lambda row: row['home_win'] if row['home_team'] == team1 else row['away_win'], axis=1).sum()
        wins_team2 = matches.apply(lambda row: row['home_win'] if row['home_team'] == team2 else row['away_win'], axis=1).sum()
        ties = matches['tie'].sum()

        # Calculate total goals using home_score and away_score columns
        goals_team1 = matches.apply(lambda row: row['home_score'] if row['home_team'] == team1 else row['away_score'], axis=1).sum()
        goals_team2 = matches.apply(lambda row: row['away_score'] if row['home_team'] == team1 else row['home_score'], axis=1).sum()

        df.loc[(df['home_team'] == team1) & (df['away_team'] == team2), 'h2h_wins_home'] = wins_team1
        df.loc[(df['home_team'] == team1) & (df['away_team'] == team2), 'h2h_wins_away'] = wins_team2
        df.loc[(df['home_team'] == team1) & (df['away_team'] == team2), 'h2h_goals_home'] = goals_team1
        df.loc[(df['home_team'] == team1) & (df['away_team'] == team2), 'h2h_goals_away'] = goals_team2
        df.loc[(df['home_team'] == team1) & (df['away_team'] == team2), 'ties'] = ties

df.loc[(df['home_team'] == "Barcelona") & (df['away_team'] == "Real Madrid")]
df.loc[(df['away_team'] == "Barcelona") & (df['home_team'] == "Real Madrid")]

"""
# It works xddd

'\nfor team1 in df[\'away_team\'].unique():\n    for team2 in df[\'home_team\'].unique():        \n        # Create a subset of matches between the two teams\n        matches = df[((df[\'home_team\'] == team1) & (df[\'away_team\'] == team2)) |\n                            ((df[\'home_team\'] == team2) & (df[\'away_team\'] == team1))]        \n\n        # Calculate wins and ties using home_win and away_win columns\n        wins_team1 = matches.apply(lambda row: row[\'home_win\'] if row[\'home_team\'] == team1 else row[\'away_win\'], axis=1).sum()\n        wins_team2 = matches.apply(lambda row: row[\'home_win\'] if row[\'home_team\'] == team2 else row[\'away_win\'], axis=1).sum()\n        ties = matches[\'tie\'].sum()\n\n        # Calculate total goals using home_score and away_score columns\n        goals_team1 = matches.apply(lambda row: row[\'home_score\'] if row[\'home_team\'] == team1 else row[\'away_score\'], axis=1).sum()\n        goals_team2 = matches.apply(lambda row: row[\'away

In [8]:
# Calculate home team total goals
home_wins1 = df.groupby('home_team')['home_win'].sum().reset_index()
home_wins2 = df.groupby('away_team')['away_win'].sum().reset_index()
home_wins2

,away_team,away_win
0,Alavés,144
1,Albacete,127
2,Alcorcón,60
3,Algeciras CF,4
4,Alicante CF,6
...,...,...
129,Valencia,368
130,Villarreal,162
131,Villarreal CF B,19
132,Xerez CD,84


In [9]:
# Calculate home team total goals
home_goals = df.groupby('home_team')['home_score'].sum().reset_index()
home_goals.columns = ['team', 'home_goals']

# Calculate away team total goals
away_goals = df.groupby('away_team')['away_score'].sum().reset_index()
away_goals.columns = ['team', 'away_goals']

# Merge the home and away goals
all_time = pd.merge(home_goals, away_goals, on='team', how='outer').fillna(0)

# Calculate total goals
all_time['total_goals'] = all_time['home_goals'] + all_time['away_goals']

# Calculate total concedes when playing at home
home_conceding = df.groupby('home_team')['away_score'].sum().reset_index()
home_conceding.columns = ['team', 'concedes_home']

# Calculate total concedes when visiting
away_conceding = df.groupby('away_team')['home_score'].sum().reset_index()
away_conceding.columns = ['team', 'concedes_away']

# Calculate total concedes
all_time_concedes = pd.merge(home_conceding, away_conceding, on='team', how='outer').fillna(0)
all_time_concedes['total_concedes'] = all_time_concedes['concedes_home'] + all_time_concedes['concedes_away']
all_time = pd.merge(all_time, all_time_concedes, on='team', how='outer').fillna(0)

home_counts = df['home_team'].value_counts()
away_counts = df['away_team'].value_counts()
total_counts = home_counts.add(away_counts, fill_value=0)
df['home_app_home'] = df['home_team'].map(home_counts)
df['home_app_total'] = df['home_team'].map(total_counts)
df['away_app_away'] = df['away_team'].map(away_counts)
df['away_app_total'] = df['away_team'].map(total_counts)

df = df.merge(all_time[['team','home_goals','total_goals','concedes_home','total_concedes']], left_on='home_team', right_on='team', how='outer').fillna(0)
df = df.merge(all_time[['team','away_goals','total_goals','concedes_away','total_concedes']], left_on='away_team', right_on='team', suffixes=('_home', '_away'), how='outer').fillna(0)
df = df.drop('team_home', axis=1)
df = df.drop('team_away', axis=1)
df = df.sort_values(by='season', ascending=True)

df['home_goals_ratio'] = df['home_goals'] / df['home_app_home']
df['away_goals_ratio'] = df['away_goals'] / df['away_app_away']
df['total_goals_home_ratio'] = df['total_goals_home'] / df['home_app_total']
df['total_goals_away_ratio'] = df['total_goals_away'] / df['away_app_total']

df['concedes_home_ratio'] = df['concedes_home'] / df['home_app_home']
df['concedes_away_ratio'] = df['concedes_away'] / df['away_app_away']
df['total_concedes_home_ratio'] = df['total_concedes_home'] / df['home_app_total']
df['total_concedes_away_ratio'] = df['total_concedes_away'] / df['away_app_total']

#df = df.drop('home_app_home', axis=1)
#df = df.drop('home_app_total', axis=1)
#df = df.drop('away_app_away', axis=1)
#df = df.drop('away_app_total', axis=1)

df['goal_diff_home'] = df['home_goals'] - df['concedes_home']
df['goal_diff_home_total'] = df['total_goals_home'] - df['total_concedes_home']
df['goal_diff_away'] = df['away_goals'] - df['concedes_away']
df['goal_diff_away_total'] = df['total_goals_away'] - df['total_concedes_away']
df['goal_diff_home_ratio'] = df['home_goals_ratio'] - df['concedes_home_ratio']
df['goal_diff_home_total_ratio'] = df['total_goals_home_ratio'] - df['total_concedes_home_ratio']
df['goal_diff_away_ratio'] = df['away_goals_ratio'] - df['concedes_away_ratio']
df['goal_diff_away_total_ratio'] = df['total_goals_away_ratio'] - df['total_concedes_away_ratio']

# Define whether it is a home or away win.
df['result'] = df.apply(lambda row: 0 if row['home_score'] > row['away_score']
                                        else 2 if row['home_score'] < row['away_score']
                                        else 1, axis=1)



# Convert 'date' column to datetime with specified format
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')

# Add a 'weekday' column with weekday names
df['weekday'] = df['date'].dt.day_name()
df['weekday_num'] = df['date'].dt.dayofweek


# Convert 'time' column to datetime (adjusting format as "hour:minute AM/PM")
# df['time_absolute'] = pd.to_datetime(df['time'], format='%I:%M %p')

# Extract the hour from the converted 'time' column
# df['hour'] = df['time_absolute'].dt.hour

df = df.drop('weekday', axis=1)


df.head(10)

,season,division,matchday,date,time,home_team,away_team,score,home_score,away_score,...,total_concedes_away_ratio,goal_diff_home,goal_diff_home_total,goal_diff_away,goal_diff_away_total,goal_diff_home_ratio,goal_diff_home_total_ratio,goal_diff_away_ratio,goal_diff_away_total_ratio,weekday_num
30982,1928-1929,1,6,2029-03-24,0,Espanyol,Real Madrid,4:0,4,0,...,1.126630,1104,-203,372,2958,0.764543,-0.070291,0.255319,1.015100,5
18929,1928-1929,1,16,2029-06-09,0,Arenas Club,Espanyol,3:0,3,0,...,1.434557,60,-118,-1307,-203,0.666667,-0.655556,-0.905125,-0.070291,5
34448,1928-1929,1,16,2029-06-09,0,Racing,Real Sociedad,6:1,6,1,...,1.370098,862,-411,-1036,296,0.668217,-0.159302,-0.725490,0.103641,5
1531,1928-1929,1,13,2029-05-26,0,Athletic,Arenas Club,2:3,2,3,...,2.305556,1685,926,-178,-118,1.156486,0.317776,-1.977778,-0.655556,5
27487,1928-1929,1,2,2029-02-17,0,Real Unión,Racing,3:1,3,1,...,1.512791,46,-144,-1273,-411,0.403509,-0.631579,-0.986822,-0.159302,5
30428,1928-1929,1,8,2029-04-07,0,Arenas Club,Real Madrid,3:2,3,2,...,1.126630,60,-118,372,2958,0.666667,-0.655556,0.255319,1.015100,5
30435,1928-1929,1,18,2029-06-23,0,Athletic,Real Madrid,2:0,2,0,...,1.126630,1685,926,372,2958,1.156486,0.317776,0.255319,1.015100,5
18937,1928-1929,1,2,2029-02-17,0,Athletic,Espanyol,9:0,9,0,...,1.434557,1685,926,-1307,-203,1.156486,0.317776,-0.905125,-0.070291,5
12680,1928-1929,1,11,2029-05-05,0,Arenas Club,CE Europa,2:2,2,2,...,1.758621,60,-118,-133,-78,0.666667,-0.655556,-1.528736,-0.448276,5
33693,1928-1929,1,6,2029-03-24,0,Arenas Club,Real Sociedad,3:0,3,0,...,1.370098,60,-118,-1036,296,0.666667,-0.655556,-0.725490,0.103641,5


In [10]:
df.corr(numeric_only=True).style.background_gradient("coolwarm", vmin=-1, vmax=1)

ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.

In [ ]:
target = "result"
features = [
    "division", "matchday", "home_app_home", "home_app_total",
    "away_app_away", "away_app_total", "home_goals", "total_goals_home", "concedes_home",
    "total_concedes_home", "away_goals", "total_goals_away", "concedes_away", "total_concedes_away",
    "home_goals_ratio", "away_goals_ratio", "total_goals_home_ratio", "total_goals_away_ratio",
    "concedes_home_ratio", "concedes_away_ratio", "total_concedes_home_ratio", "total_concedes_away_ratio",
    "goal_diff_home", "goal_diff_home_total", "goal_diff_away", "goal_diff_away_total",
    "goal_diff_home_ratio", "goal_diff_home_total_ratio", "goal_diff_away_ratio", "goal_diff_away_total_ratio",
    "weekday_num"
]
X, y = df[features], df[target]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

gbm_model = GradientBoostingRegressor()
gbm_model.fit(X_train, y_train)
gbm_y_pred = gbm_model.predict(X_test)

print(f"RMSE: {mean_squared_error(y_test, gbm_y_pred)**0.5}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, gbm_y_pred)}")
print(f"R^2: {r2_score(y_test, gbm_y_pred)}")

In [ ]:


preds_df = df.loc[y_test.index].copy()
preds_df["y_real"] = preds_df[target]  # I'm just adding this column to have both y_real and y_pred at the right of my df
preds_df["y_pred_rounded"] = gbm_y_pred.round(0)
preds_df["y_pred"] = gbm_y_pred
preds_df["err"] = preds_df["y_real"] - preds_df["y_pred"]
preds_df[["y_real","y_pred_rounded","y_pred"]].head(50)

In [ ]:
gbm_model.feature_importances_

In [ ]:
importances = pd.Series(gbm_model.feature_importances_, index=features)
importances.sort_values(ascending=True).plot(kind="barh")